In [54]:
import rasterio
import numpy as np
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine
import utm

# Configuración de la conexión a la base de datos
server = '192.168.200.31'
database = 'jmineops'
username = 'tinformacion'
password = 'Timeinlondon$'
conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(conn_str)

# Consulta SQL para obtener los datos
print("Iniciando la consulta SQL...")
query = """
SELECT TOP 500 [time]
      ,[Northing] AS j
      ,[Easting] AS i
      ,[Elevation]
      ,[speed] AS Speed
      ,[full_name]
      ,[Alarma]
  FROM [jmineops].[dbo].[T_sensors_speed_coord]
"""

data = pd.read_sql(query, engine)
print(f"Consulta SQL completada. Se obtuvieron {len(data)} registros.")

# Convertir las coordenadas UTM a latitud y longitud
def utm_to_latlon(row):
    lat, lon = utm.to_latlon(row['i'], row['j'], 13, 'N')  # Ajustar la zona y el hemisferio según sea necesario
    return pd.Series([lat, lon])

data[['latitude', 'longitude']] = data.apply(utm_to_latlon, axis=1)

# Filtrar datos para eliminar coordenadas inválidas
valid_lat_range = (18, 22)
valid_lon_range = (-107, -100)
data = data[(data['latitude'].between(*valid_lat_range)) & (data['longitude'].between(*valid_lon_range))]

# Convertir la columna 'time' a datetime
data['time'] = pd.to_datetime(data['time'])

# Eliminar filas con valores faltantes y velocidades mayores a 50 km/h
data = data.dropna(subset=['latitude', 'longitude', 'Speed', 'Alarma', 'Elevation'])
data = data[data['Speed'] <= 50]
print("Datos filtrados y preparados.")

# Ruta al archivo GeoTIFF
geotiff_path = 'Pea-Colorada-PeCo_Orto_Feb2024-57-dsm.tif'  # Cambia esta ruta al archivo GeoTIFF

# Leer y procesar el archivo GeoTIFF
print("Leyendo el archivo GeoTIFF...")
with rasterio.open(geotiff_path) as src:
    # Usar un submuestreo para reducir el tamaño de los datos cargados
    scale_factor = 20
    height, width = src.shape
    transform = src.transform

    # Leer una ventana con submuestreo
    elevation_data = src.read(
        1,
        out_shape=(height // scale_factor, width // scale_factor),
        resampling=rasterio.enums.Resampling.average
    )
    
    # Calcular las coordenadas de la malla
    x = np.linspace(transform.c, transform.c + transform.a * (width // scale_factor), width // scale_factor)
    y = np.linspace(transform.f, transform.f + transform.e * (height // scale_factor), height // scale_factor)
    x, y = np.meshgrid(x, y)

print("Lectura del GeoTIFF completada.")

# Crear un gráfico de dispersión en 3D con Plotly Express
print("Creando visualización 3D simplificada con Plotly Express...")
fig = px.scatter_3d(
    data, 
    x='longitude', 
    y='latitude', 
    z='Elevation', 
    color='Speed',
    color_continuous_scale='Viridis',
    range_color=(0, 50),
    title='Visualización 3D de Puntos de Velocidad',
    opacity=0.7
)

# Añadir una superficie simplificada de elevación
fig.add_trace(go.Surface(
    z=elevation_data, 
    x=x, 
    y=y, 
    colorscale='Viridis', 
    opacity=0.5,
    showscale=False
))

# Ajustar layout
fig.update_layout(
    scene=dict(
        xaxis_title='Longitud',
        yaxis_title='Latitud',
        zaxis_title='Elevación',
    ),
    width=800,
    height=600,
)

# Mostrar la visualización
print("Mostrando la visualización simplificada...")
fig.show()


Iniciando la consulta SQL...
Consulta SQL completada. Se obtuvieron 500 registros.
Datos filtrados y preparados.
Leyendo el archivo GeoTIFF...
Lectura del GeoTIFF completada.
Creando visualización 3D simplificada con Plotly Express...
Mostrando la visualización simplificada...


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed